In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import keras, os, cv2
from zipfile import ZipFile
%matplotlib inline

Using TensorFlow backend.


In [2]:
with ZipFile('data.zip', 'r') as zipObj:
    zipObj.extractall()

In [3]:
train_path = 'data/train/'
test_path = 'data/val/'

## BUILDING THE MODEL

In [4]:
model = keras.models.Sequential()

# Convolution/Pooling Layers
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(20,20,3)))
model.add(keras.layers.MaxPool2D())

model.add(keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(20,20,3)))
model.add(keras.layers.MaxPool2D())

# Flatten
model.add(keras.layers.Flatten())

# Dense/Dropout Layers
model.add(keras.layers.Dense(units=400, activation='relu'))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dropout(0.2))

# Classification Dense Layer
model.add(keras.layers.Dense(units=36, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 18, 18, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 400)               115600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)              

In [6]:
train_image_data_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255, 
                                                              shear_range=0.3, 
                                                              zoom_range=0.3,
                                                              width_shift_range=0.2,
                                                              height_shift_range=0.2)
test_image_data_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [7]:
train_img_gen = train_image_data_gen.flow_from_directory(directory=train_path, target_size=(20,20))
val_img_gen = test_image_data_gen.flow_from_directory(directory=test_path, target_size=(20,20), shuffle=False)

Found 864 images belonging to 36 classes.
Found 216 images belonging to 36 classes.


In [8]:
model.fit_generator(train_img_gen, steps_per_epoch=864, epochs=30, validation_data=val_img_gen, validation_steps=216)

Epoch 1/30
864/864 [==============================] - 72s 83ms/step - loss: 1.3744 - accuracy: 0.5693 - val_loss: 0.1091 - val_accuracy: 0.8885
Epoch 2/30
864/864 [==============================] - 51s 59ms/step - loss: 0.5443 - accuracy: 0.8077 - val_loss: 0.2433 - val_accuracy: 0.9491
Epoch 3/30
864/864 [==============================] - 52s 60ms/step - loss: 0.4064 - accuracy: 0.8533 - val_loss: 0.0746 - val_accuracy: 0.9536
Epoch 4/30
864/864 [==============================] - 50s 57ms/step - loss: 0.3470 - accuracy: 0.8706 - val_loss: 0.3130 - val_accuracy: 0.9535
Epoch 5/30
864/864 [==============================] - 47s 54ms/step - loss: 0.3140 - accuracy: 0.8836 - val_loss: 0.0213 - val_accuracy: 0.9634
Epoch 6/30
864/864 [==============================] - 49s 57ms/step - loss: 0.2796 - accuracy: 0.8937 - val_loss: 0.1163 - val_accuracy: 0.9535
Epoch 7/30
864/864 [==============================] - 46s 53ms/step - loss: 0.2634 - accuracy: 0.9002 - val_loss: 0.0121 - val_accuracy:

In [17]:
model.save('character_classification_new.h5')